# Using the Equities India (NSE) data-set

The python version of this sample can be found [here](https://github.com/shyams80/plutons/blob/master/docs-py/EquitiesIndiaNse.ipynb)


In [1]:
library(dbplyr)
library(dplyr)
library(odbc)
library(RPostgres)
library(plutoR)
options("scipen"=999)
source("config.R")

#initialize
equitiesIndiaNse <- EquitiesIndiaNse()


Attaching package: ‘dplyr’

The following objects are masked from ‘package:dbplyr’:

    ident, sql

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



### fetch the earliest 10 listed equity

In [2]:
print("earliest 10 listed equity")
equitiesIndiaNse$Tickers() %>%
  top_n(-10, wt = DATE_LISTING) %>%
  print(n)

[1] "earliest 10 listed equity"
# Source:   lazy query [?? x 8]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
   ISIN     SYMBOL   SERIES NAME           DATE_LISTING PAID_UP  FACE MARKET_LOT
   <chr>    <chr>    <chr>  <chr>          <date>         <dbl> <dbl>      <dbl>
 1 INE769A… AARTIIND BE     Aarti Industr… 1995-02-08         5     5          1
 2 INE117A… ABB      EQ     ABB India Lim… 1995-02-08         2     2          1
 3 INE606A… AMJLAND  EQ     Amj Land Hold… 1995-02-08         2     2          1
 4 INE034A… ARVIND   EQ     Arvind Limited 1995-02-08        10    10          1
 5 INE119A… BALRAMC… EQ     Balrampur Chi… 1995-02-08         1     1          1
 6 INE113O… BLUEBLE… BE     Blue Blends (… 1995-02-08        10    10          1
 7 INE486A… CESC     EQ     CESC Limited   1995-02-08        10    10          1
 8 INE059A… CIPLA    EQ     Cipla Limited  1995-02-08         2     2          1
 9 INE230A… EIHOTEL  EQ     EIH Limited    1995-02-08        

### fetch some "misc" info for State Bank of India

In [3]:
maxDt <- (equitiesIndiaNse$MiscInfo() %>%
  summarize(MAX_DT = max(TIME_STAMP)) %>%
  collect())$MAX_DT[[1]]

print("SBIN's latest misc info: ")
equitiesIndiaNse$MiscInfo() %>%
  filter(TIME_STAMP == maxDt & SYMBOL == 'SBIN') %>%
  print(n=Inf)

Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

[1] "SBIN's latest misc info: "
# Source:   lazy query [?? x 6]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
  SYMBOL TIME_STAMP FF_MKT_CAP_CR D2T_PCT LOWER_PX_BAND UPPER_PX_BAND
  <chr>  <date>             <dbl>   <dbl>         <dbl>         <dbl>
1 SBIN   2019-07-16       139822.    53.2          324.          396.


### fetch the market-cap decile of DHFL since we started capturing the data-set

In [4]:
print("market-cap deciles for DHFL over time:")
equitiesIndiaNse$MarketCapDecile() %>%
  filter(SYMBOL == 'DHFL') %>%
  arrange(TIME_STAMP) %>%
  print(n=Inf)

[1] "market-cap deciles for DHFL over time:"
# Source:     lazy query [?? x 3]
# Database:   Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
# Ordered by: TIME_STAMP
   SYMBOL TIME_STAMP DECILE
   <chr>  <date>      <int>
 1 DHFL   2014-08-28      1
 2 DHFL   2014-09-25      1
 3 DHFL   2014-10-30      1
 4 DHFL   2014-11-27      1
 5 DHFL   2014-12-24      1
 6 DHFL   2015-01-29      1
 7 DHFL   2015-02-26      1
 8 DHFL   2015-03-26      1
 9 DHFL   2015-04-30      1
10 DHFL   2015-05-28      1
11 DHFL   2015-06-25      1
12 DHFL   2015-07-30      1
13 DHFL   2015-08-27      1
14 DHFL   2015-09-24      1
15 DHFL   2015-10-29      1
16 DHFL   2015-11-26      1
17 DHFL   2015-12-31      1
18 DHFL   2016-01-28      1
19 DHFL   2016-02-25      1
20 DHFL   2016-03-31      1
21 DHFL   2016-04-28      1
22 DHFL   2016-05-26      1
23 DHFL   2016-06-30      1
24 DHFL   2016-07-28      1
25 DHFL   2016-08-25      1
26 DHFL   2016-09-29      1
27 DHFL   2016-10-27      1
28 DHFL   2016-11

### fetch the latest end-of-day prices for State Bank of India

In [5]:
maxDt <- (equitiesIndiaNse$EodTimeSeries() %>%
  summarize(MAX_DT = max(TIME_STAMP)) %>%
  collect())$MAX_DT[[1]]

print("latest end-of-day prices for SBIN:")
equitiesIndiaNse$EodTimeSeries() %>%
  filter(TIME_STAMP == maxDt & SYMBOL == 'SBIN') %>%
  print()

[1] "latest end-of-day prices for SBIN:"
# Source:   lazy query [?? x 9]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
  SYMBOL SERIES TIME_STAMP   HIGH    LOW   OPEN  CLOSE   LAST   VOLUME
  <chr>  <chr>  <date>      <dbl>  <dbl>  <dbl>  <dbl>  <dbl>    <dbl>
1 SBIN   EQ     2019-07-15   365    358.   364.   360.   360. 15665939
2 SBIN   N2     2019-07-15 11250. 11250  11250  11250. 11250.       80
3 SBIN   N5     2019-07-15 11470  11376. 11450  11441. 11470       191
4 SBIN   N6     2019-07-15 11250  11200  11250. 11200  11200        11


### fetch the last 10 day EOD prices for State Bank of India equity

In [6]:
print("last 10 day EOD prices for SBIN equity:")
equitiesIndiaNse$EodTimeSeries() %>%
  filter(SYMBOL == 'SBIN' & (SERIES == 'EQ' | SERIES == 'BE')) %>%
  top_n(10, wt = TIME_STAMP) %>%
  print(n=Inf)

[1] "last 10 day EOD prices for SBIN equity:"
# Source:   lazy query [?? x 9]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
   SYMBOL SERIES TIME_STAMP  HIGH   LOW  OPEN CLOSE  LAST   VOLUME
   <chr>  <chr>  <date>     <dbl> <dbl> <dbl> <dbl> <dbl>    <dbl>
 1 SBIN   EQ     2019-07-15  365   358.  364.  360.  360. 15665939
 2 SBIN   EQ     2019-07-12  367.  362.  363   364.  363. 14661748
 3 SBIN   EQ     2019-07-11  364   357   358.  363.  363  20048321
 4 SBIN   EQ     2019-07-10  362.  351.  361.  354.  355. 18681819
 5 SBIN   EQ     2019-07-09  360   352.  354.  360.  360. 16579914
 6 SBIN   EQ     2019-07-08  368.  354.  368   355.  356. 27247892
 7 SBIN   EQ     2019-07-05  374.  366   369.  371.  370. 30019347
 8 SBIN   EQ     2019-07-04  371.  366.  367.  367.  368. 19252646
 9 SBIN   EQ     2019-07-03  367.  363   365.  366.  366. 12626285
10 SBIN   EQ     2019-07-02  365   360.  362.  364.  364. 14482172


### UPL did a 1:2 bonus on 2019-07-02. see unadjusted eod vs. adjusted eod

In [7]:
startDt <- as.Date("2019-06-15")
endDt <- as.Date("2019-07-15")

print("unadjusted eod")

equitiesIndiaNse$EodTimeSeries() %>%
  filter(SYMBOL == "UPL" & TIME_STAMP >= startDt & TIME_STAMP <= endDt) %>%
  arrange(TIME_STAMP) %>%
  print(n=Inf)

print("adjusted eod")

startInt <- as.integer(format(startDt, '%Y%m%d'))
endInt <- as.integer(format(endDt, '%Y%m%d'))

equitiesIndiaNse$EodAdjustedTimeSeries() %>%
  filter(SYMBOL == "UPL" & DATE_STAMP >= startInt & DATE_STAMP <= endInt) %>%
  arrange(DATE_STAMP) %>%
  print(n=Inf)

[1] "unadjusted eod"
# Source:     lazy query [?? x 9]
# Database:   Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
# Ordered by: TIME_STAMP
   SYMBOL SERIES TIME_STAMP  HIGH   LOW  OPEN CLOSE  LAST   VOLUME
   <chr>  <chr>  <date>     <dbl> <dbl> <dbl> <dbl> <dbl>    <dbl>
 1 UPL    EQ     2019-06-17 1014.  991  1008.  995.  993   1437711
 2 UPL    EQ     2019-06-18 1004.  988   994  1000.  998   1125228
 3 UPL    EQ     2019-06-19 1006.  937. 1003   950.  953   2657234
 4 UPL    EQ     2019-06-20  941.  833   941.  870.  870  12905733
 5 UPL    EQ     2019-06-21  915.  868.  876   890.  889. 12447297
 6 UPL    EQ     2019-06-24  942.  895.  903.  937.  938   6506356
 7 UPL    EQ     2019-06-25  940.  910.  934.  935.  931.  5140843
 8 UPL    EQ     2019-06-26  964.  922.  931   958.  957   3365399
 9 UPL    EQ     2019-06-27  962.  937.  952.  944.  943.  5241680
10 UPL    EQ     2019-06-28  951   919.  946   937.  941   3292847
11 UPL    EQ     2019-07-01  957   936.  945   94

### fetch the last 10 day returns for State Bank of India

In [8]:
print("the last 10 day returns for SBIN: ")
equitiesIndiaNse$DailyReturns() %>%
  filter(SYMBOL == "SBIN") %>%
  top_n(10, wt = TIME_STAMP) %>%
  print(n=Inf)

[1] "the last 10 day returns for SBIN: "
# Source:   lazy query [?? x 3]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
   SYMBOL TIME_STAMP    VALUE
   <chr>  <date>        <dbl>
 1 SBIN   2019-07-15 -0.00976
 2 SBIN   2019-07-12  0.00110
 3 SBIN   2019-07-11  0.0254 
 4 SBIN   2019-07-10 -0.0147 
 5 SBIN   2019-07-09  0.0118 
 6 SBIN   2019-07-08 -0.0414 
 7 SBIN   2019-07-05  0.00885
 8 SBIN   2019-07-04  0.00341
 9 SBIN   2019-07-03  0.00453
10 SBIN   2019-07-02  0.00816


### fetch the last 10 corporate actions for State Bank of India

In [9]:
print("the last 10 corporate actions for SBIN: ")
equitiesIndiaNse$CorporateActions() %>%
  filter(SYMBOL == "SBIN") %>%
  top_n(10, wt = EX_DATE) %>%
  print(n=Inf)

[1] "the last 10 corporate actions for SBIN: "
# Source:   lazy query [?? x 5]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
   SYMBOL SERIES EX_DATE    PURPOSE                          WHEN_UPDATED       
   <chr>  <chr>  <date>     <chr>                            <dttm>             
 1 SBIN   EQ     2018-06-15 " ANNUAL GENERAL MEETING/ CHANG… 2018-06-08 16:01:00
 2 SBIN   EQ     2017-05-26 " DIVIDEND"                      2017-05-15 16:00:00
 3 SBIN   EQ     2017-05-26 " DIVIDEND - RS 2.60 PER SHARE … 2017-05-23 16:01:00
 4 SBIN   EQ     2016-06-03 " FINAL DIVIDEND"                2016-05-17 13:01:00
 5 SBIN   EQ     2016-06-03 " DIVIDEND - RS 2.60/- PER SHAR… 2016-06-01 17:01:00
 6 SBIN   EQ     2015-05-28 DIVIDEND                         2015-05-13 16:02:00
 7 SBIN   EQ     2015-05-28 DIVIDEND - RS 3.50/- PER SHARE … 2015-05-27 16:01:00
 8 SBIN   EQ     2014-11-20 FACE VALUE SPLIT RS.10/- TO RE.… 2014-10-29 19:01:00
 9 SBIN   EQ     2014-05-29 AGM/DIVIDEND      

### fetch the last 24 quarter EPS for State Bank of India

In [10]:
print("the last 24 quarter EPS for SBIN: ")
refs <- equitiesIndiaNse$CorporateResultsMeta() %>%
  filter(SYMBOL == 'SBIN' & IS_CONSOLIDATED == 0 & PERIOD %like% '%quarter') %>%
  top_n(24, wt=PERIOD_END) %>%
  select(REF_ID, PERIOD_END, PERIOD) %>%
  collect()

print(refs)

for(i in 1:nrow(refs)){
  ref <- refs[i,]
  print(ref)

  rid <- ref$REF_ID[1]
  equitiesIndiaNse$CorporateResults() %>%
    filter(REF_ID == rid & KEY %like% '%diluted%before%') %>%
    print(n=Inf)
}

[1] "the last 24 quarter EPS for SBIN: "
# A tibble: 24 x 3
   REF_ID               PERIOD_END PERIOD        
   <integr64>           <date>     <chr>         
 1 -9223372036854630265 2019-03-31 Fourth Quarter
 2 -9223372036854634303 2018-12-31 Third Quarter 
 3 -9223372036854635315 2018-09-30 Second Quarter
 4 -9223372036854635235 2018-06-30 First Quarter 
 5 -9223372036854639975 2018-03-31 Fourth Quarter
 6 -9223372036854649838 2017-12-31 Third Quarter 
 7 -9223372036854652758 2017-09-30 Second Quarter
 8 -9223372036854666804 2017-06-30 First Quarter 
 9 -9223372036854672373 2017-03-31 Fourth Quarter
10 -9223372036854674592 2016-12-31 Third Quarter 
# … with 14 more rows
# A tibble: 1 x 3
  REF_ID               PERIOD_END PERIOD        
  <integr64>           <date>     <chr>         
1 -9223372036854630265 2019-03-31 Fourth Quarter
# Source:   lazy query [?? x 7]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
  REF_ID          H1    H2    H3    H4    KEY           

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)